In [4]:
import pandas as pd 
import os 
import numpy as np

In [2]:
# reading files from the PV inverter
names_files = os.listdir('./data-base/Dados Sistema FV - Inversor FV') 
names_files_tamanho = len(names_files)                      

primeiro = 1 
for i in range(0, names_files_tamanho): # lopping to read the files
    if primeiro == 1: 
        address = './data-base/Dados Sistema FV - Inversor FV/' + names_files[i-1] # address of the file to be read
        df_file_FV = pd.read_csv(address, sep = ",", index_col=None, header=0, encoding='latin-1')
        
        if 'Energia | Primo 6.0-1 (1)' in df_file_FV.columns:
            df_file_FV.drop([0], axis=0, inplace=True)
            df_file_FV["Data e horário"] = pd.to_datetime(df_file_FV["Data e horário"], format = '%d.%m.%Y %H:%M')
            df_FV = df_file_FV[['Data e horário' , 'Energia | Primo 6.0-1 (1)']] # saves in the new file only the desired columns
            primeiro = 0
            
    else: # second iteration to increment the file with data from the other files in the folder
        address = './data-base/Dados Sistema FV - Inversor FV/' + names_files[i-1]
        df_temp = pd.read_csv(address, sep = ",", index_col=None, header=0, encoding='latin-1') # df_temp = temporary file for storing the file to be opened in that iteration
        
        if 'Energia | Primo 6.0-1 (1)' in df_temp.columns:
            df_temp.drop([0], axis=0, inplace=True)
            df_temp["Data e horário"] = pd.to_datetime(df_temp["Data e horário"], format = '%d.%m.%Y %H:%M')
            df_temp2 = df_temp[['Data e horário' , 'Energia | Primo 6.0-1 (1)']] 
            df_FV = df_FV.append(df_temp2)
        else:
            continue
    continue
    
df_FV = df_FV.rename(columns = {'Energia | Primo 6.0-1 (1)': 'FV - Energia'}, inplace = False)
df_FV['FV - Energia'] = df_FV['FV - Energia'].astype(float)
df_FV = df_FV.rename(columns = {'Data e horário': 'Time'}, inplace = False)
df_FV['Time'] = df_FV['Time'] + pd.Timedelta(minutes = -20)
df_FV['FV - Energia'] = df_FV['FV - Energia']*(1/1000)

df_FV = df_FV.dropna() #Filters PV Inverter: take empty values

C:\Users\amand\AppData\Local\Temp\ipykernel_7668\3316179120.py:20: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temp = pd.read_csv(address, sep = ",", index_col=None, header=0, encoding='latin-1') # df_temp = arquivo temporário para armazenamento do arquivo a ser aberto naquela iteração
C:\Users\amand\AppData\Local\Temp\ipykernel_7668\3316179120.py:20: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temp = pd.read_csv(address, sep = ",", index_col=None, header=0, encoding='latin-1') # df_temp = arquivo temporário para armazenamento do arquivo a ser aberto naquela iteração
C:\Users\amand\AppData\Local\Temp\ipykernel_7668\3316179120.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_F

In [3]:
# reading schneider meter file - the steps are the same PV inverter

names_files = os.listdir('./data-base/Dados Bluelog/Sensores') 
names_files_tamanho = len(names_files)                      

primeiro = 1 
for i in range(0, names_files_tamanho): 
    if primeiro == 1: 
        address = './data-base/Dados Bluelog/Sensores/' + names_files[i-1] 
        df_Schneider_temp1 = pd.read_csv(address, sep = ";", index_col=None, header=0, dtype={"Schneider Electric iEM3155 19194118:Power AC phase 1 W": float}) 
        
        if 'Schneider Electric iEM3155 19194118:Power AC W' in df_Schneider_temp1.columns: 
            df_Schneider_temp1["Time"] = pd.to_datetime(df_Schneider_temp1["Time"], format = "%Y/%m/%d %H:%M:%S")
            df_Schneider = df_Schneider_temp1[['Time' , 'Schneider Electric iEM3155 19194118:Power AC phase 1 W', 'Schneider Electric iEM3155 19194118:Power AC phase 2 W', 'Schneider Electric iEM3155 19194118:Power AC phase 3 W']] 
            primeiro = 0
        
    else:
        address = './data-base/Dados Bluelog/Sensores/' + names_files[i-1]
        df_Schneider_temp2 = pd.read_csv(address, sep = ";", index_col=None, header=0) 
        
        if 'Schneider Electric iEM3155 19194118:Power AC W' in df_Schneider_temp2.columns:
            df_Schneider_temp2["Time"] = pd.to_datetime(df_Schneider_temp2["Time"], format = "%Y/%m/%d %H:%M:%S") 
            df_Schneider_temp3 = df_Schneider_temp2[['Time' , 'Schneider Electric iEM3155 19194118:Power AC phase 1 W', 'Schneider Electric iEM3155 19194118:Power AC phase 2 W', 'Schneider Electric iEM3155 19194118:Power AC phase 3 W']] 
            df_Schneider = df_Schneider.append(df_Schneider_temp3) 
        else:
            continue
    continue
    
df_Schneider['Energia - Phase 1'] = df_Schneider['Schneider Electric iEM3155 19194118:Power AC phase 1 W'].astype(float)
df_Schneider['Energia - Phase 2'] = df_Schneider['Schneider Electric iEM3155 19194118:Power AC phase 2 W'].astype(float)
df_Schneider['Energia - Phase 2'] = df_Schneider['Schneider Electric iEM3155 19194118:Power AC phase 2 W'].astype(float)

df_Schneider['Energia - Phase 1'] = df_Schneider['Schneider Electric iEM3155 19194118:Power AC phase 1 W']*(1/60000) # converts from W to kWh
df_Schneider['Energia - Phase 2'] = df_Schneider['Schneider Electric iEM3155 19194118:Power AC phase 2 W']*(1/60000) # converts from W to kWh
df_Schneider['Energia - Phase 3'] = df_Schneider['Schneider Electric iEM3155 19194118:Power AC phase 3 W']*(-1/60000) # converts from W to kWh

df_Schneider['Schneider - Energia 3F'] = df_Schneider['Energia - Phase 1'] + df_Schneider['Energia - Phase 2'] + df_Schneider['Energia - Phase 3']

del df_Schneider['Schneider Electric iEM3155 19194118:Power AC phase 1 W']
del df_Schneider['Schneider Electric iEM3155 19194118:Power AC phase 2 W']
del df_Schneider['Schneider Electric iEM3155 19194118:Power AC phase 3 W']

del df_Schneider['Energia - Phase 1']
del df_Schneider['Energia - Phase 2']
del df_Schneider['Energia - Phase 3']

df_Schneider = df_Schneider.resample('5T', on='Time').sum()

indexNames = df_Schneider[df_Schneider['Schneider - Energia 3F'] == 0 ].index # Filters - schneider: removing the zeros
df_Schneider.drop(indexNames , inplace=True) # Delete these row indexes from dataFrame

C:\Users\amand\AppData\Local\Temp\ipykernel_7668\925439370.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_Schneider = df_Schneider.append(df_Schneider_temp3) # incremento do arquivo
C:\Users\amand\AppData\Local\Temp\ipykernel_7668\925439370.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_Schneider = df_Schneider.append(df_Schneider_temp3) # incremento do arquivo
C:\Users\amand\AppData\Local\Temp\ipykernel_7668\925439370.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_Schneider = df_Schneider.append(df_Schneider_temp3) # incremento do arquivo
C:\Users\amand\AppData\Local\Temp\ipykernel_7668\925439370.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a futu

In [ ]:
# Outlier filter - Interquartile Range Method (IQR)

def remove_outlier_IQR(df): #funcao do IQR
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3-Q1
    df_final = df[~((df<(Q1-1.5*IQR)) | (df>(Q3+1.5*IQR)))]
    return df_final

# --------------------------------------------------------------------------------------------------------------------
# filtro outliers - PV

df_outlier_removed = remove_outlier_IQR(df_Schneider['Schneider - Energia 3F'])
df_outlier_removed = pd.DataFrame(df_outlier_removed)
ind_diff_Schneider = df_Schneider.index.difference(df_outlier_removed.index)

for i in range(0, len(ind_diff_Schneider),1):
    df_final = df_Schneider.drop([ind_diff_Schneider[i]])
    df_Schneider = df_final

In [4]:

# reading SMA meter file - the steps are the same PV inverte

names_files = os.listdir('./data-base/Dados Bluelog/Inversores_e_medidores') 
names_files_tamanho = len(names_files)                    

primeiro = 1 
for i in range(0, names_files_tamanho): 
    if primeiro == 1: 
        address = './data-base/Dados Bluelog/Inversores_e_medidores/' + names_files[i-1] 
        df_file_SMA = pd.read_csv(address, sep = ";", index_col=None, header=0) 
        
        if 'SMA Solar Inverter 3007145318 1:Power AC W' in df_file_SMA.columns: 
            df_file_SMA["Time"] = pd.to_datetime(df_file_SMA["Time"], format = "%Y/%m/%d %H:%M:%S")
            df_SMA = df_file_SMA[['Time' , 'SMA Solar Inverter 3007145318 1:Power AC W']] 
            primeiro = 0
            
    else: 
        address = './data-base/Dados Bluelog/Inversores_e_medidores/' + names_files[i-1]
        df_temp = pd.read_csv(address, sep = ";", index_col=None, header=0) 
        
        if 'SMA Solar Inverter 3007145318 1:Power AC W' in df_temp.columns:
            df_temp["Time"] = pd.to_datetime(df_temp["Time"], format = "%Y/%m/%d %H:%M:%S") 
            df_temp2 = df_temp[['Time' , 'SMA Solar Inverter 3007145318 1:Power AC W']] 
            df_SMA = df_SMA.append(df_temp2) 
        else:
            continue
    continue

df_SMA['SMA Solar Inverter 3007145318 1:Power AC W'] = df_SMA['SMA Solar Inverter 3007145318 1:Power AC W'].astype(float)
df_SMA['SMA - Energia'] = df_SMA['SMA Solar Inverter 3007145318 1:Power AC W']*(1/60000) # converts from W to kWh
del df_SMA['SMA Solar Inverter 3007145318 1:Power AC W']
df_SMA = df_SMA.reset_index(drop=True)

df_SMA_day = df_SMA
df_SMA_day['SMA - Energia ABS - Day'] = abs(df_SMA['SMA - Energia']) # transforms to absolute value
df_SMA_day["Time"] = pd.to_datetime(df_SMA_day["Time"], format = "%Y/%m/%d %H:%M:%S") 
df_SMA_day = df_SMA_day.resample('1d', on='Time').sum() # transforms into day, adding up all the values of the day
df_SMA_day = df_SMA_day.reset_index(drop=False)

# SMA: sum of the absolute energy of the days, if equal to 0 it takes away the day 
indexNames = df_SMA_day[df_SMA_day['SMA - Energia ABS - Day'] == 0 ].index
df_SMA_day.drop(indexNames , inplace=True) # Delete these row indexes from dataFrame
df_SMA_day['just_date'] = df_SMA_day['Time'].dt.date

df_SMA['just_date'] = df_SMA['Time'].dt.date
df_SMA = df_SMA.rename(columns = {'SMA - Energia': 'SMA - Energia - Seg'}, inplace = False)

df_SMA_filter = df_SMA.merge(df_SMA_day, left_on = 'just_date', right_on = 'just_date')
df_SMA_filter = df_SMA_filter.reset_index(drop=False)
df_SMA_filter = df_SMA_filter.rename(columns = {'Time_x': 'Time'}, inplace = False)
df_SMA_filter = df_SMA_filter.resample('5T', on='Time').sum()
df_SMA_filter = df_SMA_filter.reset_index(drop=False)

del df_SMA_filter['SMA - Energia']
del df_SMA_filter['SMA - Energia ABS - Day_x']
del df_SMA_filter['SMA - Energia ABS - Day_y']
del df_SMA_filter['index']

C:\Users\amand\AppData\Local\Temp\ipykernel_7668\2176316163.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_SMA = df_SMA.append(df_temp2) # incremento do arquivo
C:\Users\amand\AppData\Local\Temp\ipykernel_7668\2176316163.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_SMA = df_SMA.append(df_temp2) # incremento do arquivo
C:\Users\amand\AppData\Local\Temp\ipykernel_7668\2176316163.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_SMA = df_SMA.append(df_temp2) # incremento do arquivo
C:\Users\amand\AppData\Local\Temp\ipykernel_7668\2176316163.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_SMA = df_SMA.appen

In [5]:
# add

df_Schneider_FV = pd.merge(df_Schneider, df_FV, on = 'Time', how = 'outer')
df_Schneider_FV_SMA = pd.merge(df_Schneider_FV, df_SMA_filter, on = 'Time', how = 'outer')
df_Schneider_FV_SMA['FV - Energia'] = df_Schneider_FV_SMA['FV - Energia'] * (-1) 

df_Schneider_FV_SMA = df_Schneider_FV_SMA.resample('5T', on='Time').sum()

#schneider: remove the zeros
indexNames_2 = df_Schneider_FV_SMA[df_Schneider_FV_SMA['Schneider - Energia 3F'] == 0 ].index
df_Schneider_FV_SMA.drop(indexNames_2 , inplace=True) # Delete these row indexes from dataFrame

df_Schneider_FV_SMA = df_Schneider_FV_SMA[['Schneider - Energia 3F', 'SMA - Energia - Seg', 'FV - Energia']] # changes the order of the columns
# --------------------------------------------------------------------------------------------------------------------

In [ ]:
# --------------------------------------------------------------------------------------------------------------------
# outlier filter - Interquartile Range Method (IQR)

def remove_outlier_IQR(df): 
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3-Q1
    df_final = df[~((df<(Q1-1.5*IQR)) | (df>(Q3+1.5*IQR)))]
    return df_final

# --------------------------------------------------------------------------------------------------------------------
# outlier filter - Energy meter

df_outlier_removed = remove_outlier_IQR(df_Schneider_FV_SMA['Schneider - Energia 3F'])
df_outlier_removed = pd.DataFrame(df_outlier_removed)
ind_diff_Schneider = df_Schneider_FV_SMA.index.difference(df_outlier_removed.index)

for i in range(0, len(ind_diff_Schneider),1):
    df_final = df_Schneider_FV_SMA.drop([ind_diff_Schneider[i]])
    df_Schneider_FV_SMA = df_final

df_Schneider_FV_SMA.to_csv("./data-saida/dados_projeto_residencia-teste-amanda.csv") 